# Amadeus API example

This notebook provides an example about the use of API to retrieve price data.

Specifically, it uses [Amadeus Flight API](https://developers.amadeus.com/self-service/category/flights) to retrieve flight prices. At the date of writing (september 2023) Amadeus provides up to 2000 free calls for flight search every month. You will need to register on Amadeus website in order to get your API key, necessary to retrieve flight price data.

You can find more information about getting an Amadeus API key in their [FAQ](https://developers.amadeus.com/support/faq/?page=1&count=50), under "How do I get my Self-Service API key?".

If you use the Amadeus test environment, you may not be able to retrieve data on less common routes, as the data they make available for the test environment is a subset of the total data available. We suggest you to to use production API keys when retrieving data on specific routes.

You can run this notebook on your own environment, provided you install the Python packages listed in the requirements.txt file. You can also run this notebook on a cloud environment like Google Colab. In this case, you will have to follow specific directions in the cells below, uncommenting code as required. The minimum version of Python tested with this notebook is 3.9, we suggest at least 3.10 or newer.





There are two complementary files needed to run this notebook:

*   amadeus.ini: You should edit this file with your own API key details
*   flight_list.csv: You should insert in this files the routes for which you want to retrieve prices.

The notebook is structured assuming you have those two files in the same folder as the notebook. In case you prefer a different structure, you should adjust the paths to those files accordingly in the cells below.

The file flight_list.csv has the following columns:

*   origin: IATA 3-letter airport code for flight departure
*   destination: IATA 3-letter airport code for flight arrival
*   stay_days: How many days between outbound and return flight
*   advance1: How many days after the current date you want to depart (1st search)
*   advance2: How many days after the current date you want to depart (2nd search)

You can search IATA 3-letter codes on their [official website](https://www.iata.org/en/publications/directories/code-search/).

There are two searches at different time horizons (advance1 and advance2), as this is common practice in several National Statistic Organization. If you only want to retrieve prices with a single time horizon, you can modify the script accordingly.


In [ ]:
# Install the Python library from https://pypi.org/project/amadeus

# Uncomment code below if running on Google Colab

#try:
#  from amadeus import Client, ResponseError
#except:
#  !pip install amadeus >> \dev\null
#  from amadeus import Client, ResponseError

In [ ]:
# Uncomment code below if running on Google Colab

#from google.colab import drive
#drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
from configparser import ConfigParser
import time
from datetime import datetime, timedelta
from amadeus import Client, ResponseError
import pandas as pd

In [ ]:
config = ConfigParser()
config.read('amadeus.ini')

# Flight prices

In [ ]:
# Instantiate client with secrets from config file
amadeus = Client(
    client_id=config["DEFAULT"]["amadeus_api_key"],
    client_secret=config["DEFAULT"]["amadeus_api_secret"]
)


In [ ]:
# read list of origins and destinations
routes_df = pd.read_csv("flight_list.csv")

### Function

In [ ]:
def get_flight_prices(origin: str, destination: str, advance_days: int, stay_days: int, limit: int = 20) -> pd.DataFrame:
  """Function to get flight prices between origin and destination
  for a specific flight date. Return flights only. Maximum 20 options.

  Parameters:
  origin (str): departure airport code, 3 capital letters
  destination (str): destination airport code, 3 capital letters
  advance_days (int): days before departure date
  stay_days (int): days between departure and return date
  limit (int): maximum number of flight offer to retrieve.

  Returns:
  pandas Dataframe with flight information and prices
  """
  flight_date = (datetime.now() + timedelta(days=advance_days)).strftime("%Y-%m-%d")
  return_date = (datetime.now() + timedelta(days=(advance_days + stay_days))).strftime("%Y-%m-%d")
  try:
    # Individual flights
    flights = amadeus.shopping.flight_offers_search.get(originLocationCode=origin,
                                                        destinationLocationCode=destination,
                                                        departureDate=flight_date,
                                                        returnDate=return_date,
                                                        adults=1,
                                                        nonStop="true",  # removing the non-stop requirement may raise errors.
                                                        travelClass="ECONOMY",
                                                        currencyCode="EUR",
                                                        max=limit).data
    # Break the list in chunks of 5 to avoid errors
    flight_lists = [flights[i:i + 5] for i in range(0, len(flights), 5)]
    response_all_flights = []
    for f in flight_lists:
      time.sleep(1)
      # Try except block to overcome authentication issues. Maybe related to test environment
      try:
        response_all_flights.extend(amadeus.shopping.flight_offers.pricing.post(f).data.get("flightOffers", []))
      except:
        continue
    # Price analysis
    analysis = amadeus.analytics.itinerary_price_metrics.get(
      originIataCode=origin,
      destinationIataCode=destination,
      departureDate=flight_date,
      currencyCode="EUR")
    # Analysis data seems to have issues with some route. Probaly related to limited data in the test environment
    try:
      analysis_dict = {level.get("quartileRanking"): level.get("amount") for level in analysis.data[0].get("priceMetrics")}
    except:
      analysis_dict = {}
    results = []
    for flight in response_all_flights:
      results.append(
          {
              "origin": origin,
              "destination": destination,
              "flight_date": flight_date,
              "departure_time": flight.get("itineraries", [{}])[0].get("segments", [{}])[0].get("departure", {}).get("at")[-8:],
              "return_date": return_date,
              "return_time": flight.get("itineraries", [{}])[-1].get("segments", [{}])[0].get("departure", {}).get("at")[-8:],
              "acquisition_date": datetime.now().strftime("%Y-%m-%d"),
              "source": flight.get("source"),
              "price": flight.get("price", {}).get("grandTotal"),
              "net_price": flight.get("price", {}).get("base"),
              "currency": flight.get("price", {}).get("currency"),
              "carrier_outbound": flight.get("itineraries", [{}])[0].get("segments", [{}])[0].get("carrierCode"),
              "flight_outbound": flight.get("itineraries", [{}])[0].get("segments", [{}])[0].get("number"),
              "carrier_inbound": flight.get("itineraries", [{}])[-1].get("segments", [{}])[0].get("carrierCode"),
              "flight_inbound": flight.get("itineraries", [{}])[-1].get("segments", [{}])[0].get("number"),
              **analysis_dict
          }
      )
    results_df = pd.DataFrame(results)
    return results_df

  except ResponseError as error:
    raise error
  except Exception as error:  # General catch for all other exceptions
    raise error



Test call the function with sample parameters

In [ ]:
data_df = get_flight_prices(origin = "FCO", destination = "JFK", advance_days = 10, stay_days = 7, limit=20)

In [ ]:
data_df

,origin,destination,flight_date,departure_time,return_date,return_time,acquisition_date,source,price,net_price,currency,carrier_outbound,flight_outbound,carrier_inbound,flight_inbound
0,FCO,JFK,2023-08-04,10:30:00,2023-08-11,17:00:00,2023-07-25,GDS,627.38,285.00,EUR,IB,4871,IB,4872
1,FCO,JFK,2023-08-04,17:15:00,2023-08-11,22:34:00,2023-07-25,GDS,621.38,255.00,EUR,AY,4248,AY,4273
2,FCO,JFK,2023-08-04,10:30:00,2023-08-11,17:00:00,2023-07-25,GDS,621.38,255.00,EUR,AY,4189,AY,4190
3,FCO,JFK,2023-08-04,17:15:00,2023-08-11,17:00:00,2023-07-25,GDS,621.38,255.00,EUR,AY,4248,AY,4190
4,FCO,JFK,2023-08-04,10:30:00,2023-08-11,22:34:00,2023-07-25,GDS,621.38,255.00,EUR,AY,4189,AY,4273


## Full loop

In [ ]:
results = []
for route in routes_df.itertuples():
  # First advance window
  temp_df = get_flight_prices(
      origin = route.origin,
      destination = route.destination,
      advance_days = route.advance1,
      stay_days = route.stay_days,
      limit = 20)
  results.append(temp_df.copy())
  # Second advance window
  temp_df = get_flight_prices(
      origin = route.origin,
      destination = route.destination,
      advance_days = route.advance2,
      stay_days = route.stay_days,
      limit = 20)
  results.append(temp_df.copy())

In [ ]:
data_df = pd.concat(results, ignore_index=True)

In [ ]:
# Save file with current date in the name
data_df.to_csv("flight_prices_{}.csv".format(datetime.now().strftime("%Y-%m-%d")), index=False)

In [ ]:
data_df.shape

(138, 20)

In [ ]:
data_df.head()

,origin,destination,flight_date,departure_time,return_date,return_time,acquisition_date,source,price,net_price,currency,carrier_outbound,flight_outbound,carrier_inbound,flight_inbound,MINIMUM,FIRST,MEDIUM,THIRD,MAXIMUM
0,FCO,PMO,2023-07-14,08:20:00,2023-07-21,15:10:00,2023-07-04,GDS,128.06,32.00,EUR,AZ,1777,AZ,1792,56.13,251.87,290.62,331.79,411.11
1,FCO,PMO,2023-07-14,08:20:00,2023-07-21,14:05:00,2023-07-04,GDS,128.06,32.00,EUR,AZ,1777,AZ,1766,56.13,251.87,290.62,331.79,411.11
2,FCO,PMO,2023-07-14,08:20:00,2023-07-21,11:55:00,2023-07-04,GDS,128.06,32.00,EUR,AZ,1777,AZ,1784,56.13,251.87,290.62,331.79,411.11
3,FCO,PMO,2023-07-14,08:20:00,2023-07-21,06:10:00,2023-07-04,GDS,128.06,32.00,EUR,AZ,1777,AZ,1778,56.13,251.87,290.62,331.79,411.11
4,FCO,PMO,2023-07-14,17:10:00,2023-07-21,06:10:00,2023-07-04,GDS,128.06,32.00,EUR,AZ,1795,AZ,1778,56.13,251.87,290.62,331.79,411.11
